In [44]:
import shutil, time

In [45]:
devs = %mpy --list
ports = devs.fields(0) # list of ports
# build a list of dicts with info for each port and attached board 
dev_info = []
for p in ports:
    info = %mpy --select {p} --info
    if info:
        dev_info.append(info)

target_version = "1.22.0"
# filter 
# dev_info = [d for d in dev_info if d["version"].rstrip(".") != target_version]
dev_info

[{'port': 'esp32',
  'build': '',
  'arch': 'xtensawin',
  'family': 'micropython',
  'board': 'Generic ESP32S3 module with ESP32S3',
  'cpu': 'ESP32S3',
  'version': '1.22.0.',
  'mpy': 'v6.2',
  'ver': 'v1.22.0.',
  'serial_port': 'COM8'}]

In [46]:
from pathlib import Path

PORT_FWTYPES = {
    "stm32": ".hex",
    "esp32": ".bin",
    "rp2": ".uf2",
    "samd": ".uf2",
}


def find_firmware(version: str, *, board: str = "*", port: str):
    fw_folder = Path.cwd() / "firmware" / port
    if not fw_folder.exists():
        print(f"Folder {fw_folder} does not exist")
        return None
    v_str = f"v{version}"
    ext = PORT_FWTYPES[port]
    fw_files = list(fw_folder.glob(f"{board}-{v_str}{ext}"))
    if not fw_files:
        print(f"No firmware files found for {v_str}")
        return None
    fw_files.sort()
    yield from fw_files


# for fw in find_firmware("1.22.0", port="samd", board="SEEED_WIO_*"):
#     print(fw)
# for fw in find_firmware("1.22.0", port="esp32", board="ESP32_GENERIC-SPIRAM"):
#     print(fw)

In [47]:
import gc
import os


LIBS = ["../src/stubber/data"]


def file_exists(filename: str):
    try:
        if os.stat(filename)[0] >> 14:
            return True
        return False
    except OSError:
        return False


def read_boardname(info, desc: str = ""):
    found = False
    for filename in [d + "/board_info.csv" for d in LIBS]:
        # print("look up the board name in the file", filename)
        if file_exists(filename):
            descr = desc or info["board"].strip()
            print("searching info file: {} for: '{}' ".format(filename, descr))
            if find_board(info, descr, filename):
                found = True
                break
    if not found:
        print("Board not found, guessing board name")
        descr = desc or info["board"].strip()
        if "with " + info["cpu"].upper() in descr:
            # remove the with cpu part
            descr = descr.split("with " + info["cpu"].upper())[0].strip()
        info["board"] = descr
    info["board"] = info["board"].replace(" ", "_")
    gc.collect()


def find_board(info: dict, descr: str, filename: str):
    # find a board based on its description
    short_hit = ""
    lines = []
    pos = descr.rfind(" with")
    if pos != -1:
        short_descr = descr[:pos].strip()
    else:
        short_descr = ""
    try:
        with open(filename, "r") as file:
            lines = file.readlines()
        for line in lines:
            line = line.strip()
            if line.startswith("#"):
                continue
            _descr = line.split(",")[0].strip()
            _board = line.split(",")[1].strip()
            if _descr == descr:
                info["board"] = _board
                return True
            elif short_descr and _descr == short_descr:
                if "with" in short_descr:
                    # Good enough - no need to trawl the entire file
                    info["board"] = _board
                    return True
                # good enough if not found in the rest of the file (but slow)
                short_hit = _board
        if short_hit:
            info["board"] = short_hit
            return True
        return False
    finally:
        del lines
        gc.collect()


def find_board_old(info: dict, descr: str, filename: str, short_descr: str):
    "Find the board in the provided board_info.csv file"
    short_hit = ""
    with open(filename, "r") as file:
        # ugly code to make testable in python and micropython
        # TODO: This is VERY slow on micropython whith MPREMOTE mount on esp32 (2-3 minutes to read file)
        while 1:
            line = file.readline()
            if not line:
                break
            descr_, board_ = line.split(",")[0].strip(), line.split(",")[1].strip()
            if descr_ == descr:
                info["board"] = board_
                return True
            elif short_descr and descr_ == short_descr:
                if "with" in short_descr:
                    # Good enough - no need to trawl the entire file
                    info["board"] = board_
                    return True
                # good enough if not found in the rest of the file (but slow)
                short_hit = board_
    if short_hit:
        info["board"] = short_hit
        return True
    return False

In [48]:
mcu = dev_info.pop()
read_boardname(mcu)
fw_file = find_firmware(target_version, port=mcu["port"], board=mcu["board"]).__next__()
fw_file, mcu

searching info file: ../src/stubber/data/board_info.csv for: 'Generic ESP32S3 module with ESP32S3' 


(WindowsPath('c:/develop/MyPython/micropython-stubber/scripts/firmware/esp32/ESP32_GENERIC_S3-v1.22.0.bin'),
 {'port': 'esp32',
  'build': '',
  'arch': 'xtensawin',
  'family': 'micropython',
  'board': 'ESP32_GENERIC_S3',
  'cpu': 'ESP32S3',
  'version': '1.22.0.',
  'mpy': 'v6.2',
  'ver': 'v1.22.0.',
  'serial_port': 'COM8'})

In [49]:
from typing import Dict, Optional
import psutil
MCUInfo = Dict[str,str]
def flash_uf2(mcu:MCUInfo, fw_file:Optional[Path]= None) -> Optional[MCUInfo]:
    if not PORT_FWTYPES[mcu["port"]] in [".uf2"]:
        print(f"UF2 not supported on {mcu['board']} on {mcu['serial_port']}")
        return None

    print(f"Entering bootloader on {mcu['board']} on {mcu['serial_port']}")
    %mpy --select {mcu['serial_port']}
    %mpy --bootloader # TODO: add this to micropython-magic

    destination = ""
    wait = 5
    while not destination and wait > 0:
        print(f"Waiting for mcu to mount as a drive : {wait} seconds left")
        drives = [drive.device for drive in psutil.disk_partitions()]
        for drive in drives:
            if Path(drive, "INFO_UF2.TXT").exists():
                destination = drive
                break
            time.sleep(1)
        wait -= 1
    if not destination or not Path(destination).exists() or not Path(destination, "INFO_UF2.TXT").exists():
        print("Board is not in bootloader mode")
        return None
    else:
        print("Board is in bootloader mode")
        fw_file = fw_file or find_firmware(target_version, port=mcu["port"], board=mcu["board"]).__next__()
        print(f"Copying {fw_file} to {destination}")
        shutil.copy(fw_file, destination)
        print("Done copying, resetting the board and wait for it to restart")
        time.sleep(5)
        new_info:MCUInfo = %mpy --select {mcu['serial_port']} --info
        return new_info

In [50]:
def flash_esp32(mcu:MCUInfo, fw_file:Optional[Path]= None,*, erase_flash:bool=True) -> Optional[MCUInfo]:
    if not mcu["port"] in ["esp32"]:
        print(f"ESP32 not supported on {mcu['board']} on {mcu['serial_port']}")
        return None

    fw_file = fw_file or find_firmware(target_version, port=mcu["port"], board=mcu["board"]).__next__()
    print(f"Flashing {fw_file} on {mcu['board']} on {mcu['serial_port']}")
    baud_rate = str(2000000)
    if mcu["cpu"].upper()== "ESP32":
        if erase_flash:
            !esptool --chip esp32 --port {mcu["serial_port"]} erase_flash
        !esptool --chip esp32 --port {mcu["serial_port"]} -b {baud_rate} write_flash -z 0x1000 {fw_file}
    elif mcu["cpu"].upper() == "ESP32S3":
        if erase_flash:
            !esptool --chip esp32s3 --port {mcu["serial_port"]} erase_flash
        !esptool --chip esp32s3 --port {mcu["serial_port"]} -b {baud_rate} write_flash -z 0x0 {fw_file}

    print("Done flashing, resetting the board and wait for it to restart")
    time.sleep(5)
    new_info:MCUInfo = %mpy --select {mcu['serial_port']} --info
    return new_info


In [51]:
print(mcu)
new = None
if mcu["port"] in ["samd","rp2"]:
    new = flash_uf2(mcu)
elif mcu["port"] in ["esp32"]:
    new = flash_esp32(mcu, erase_flash=False)
if new:
    read_boardname(new)
    print(new)
    mcu = new


{'port': 'esp32', 'build': '', 'arch': 'xtensawin', 'family': 'micropython', 'board': 'ESP32_GENERIC_S3', 'cpu': 'ESP32S3', 'version': '1.22.0.', 'mpy': 'v6.2', 'ver': 'v1.22.0.', 'serial_port': 'COM8'}
Flashing c:\develop\MyPython\micropython-stubber\scripts\firmware\esp32\ESP32_GENERIC_S3-v1.22.0.bin on ESP32_GENERIC_S3 on COM8
esptool.py v4.6.2
Serial port COM8
Connecting....
Chip is ESP32-S3 (revision v0.1)
Features: WiFi, BLE
Crystal is 40MHz
MAC: f4:12:fa:81:3b:8c
Uploading stub...
Running stub...
Stub running...
Changing baud rate to 2000000
Changed.
Configuring flash size...
Flash will be erased from 0x00000000 to 0x0018dfff...
Compressed 1628240 bytes to 1066966...
Writing at 0x00000000... (1 %)
Writing at 0x0001248d... (3 %)
Writing at 0x0001a1cd... (4 %)
Writing at 0x00021df4... (6 %)
Writing at 0x000283d4... (7 %)
Writing at 0x00032609... (9 %)
Writing at 0x00041899... (10 %)
Writing at 0x000476af... (12 %)
Writing at 0x00051717... (13 %)
Writing at 0x0005973a... (15 %)
Wri

In [ ]:
PORT_FWTYPES[mcu["port"]]